In [1]:
import os
os.makedirs('output', exist_ok=True)

import sys
sys.path.append('../')

from utils import load_walksheds, overlay_wks
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read shapefiles
wksp5, wksp75 = load_walksheds()
all_boundaries= gpd.read_file('../../Data/Merge Boundaries/MD_DC_VA/Block MD_DC_VA.shp') ###Block data
SLD_data = gpd.read_file("../../Data/SmartLocationDatabaseV3/SmartLocationDatabase.gdb")
Block_MD = gpd.read_file('../../Data/GDB states/tlgdb_2021_a_24_md.gdb')
Block_DC = gpd.read_file('../../Data/GDB states/tlgdb_2021_a_11_dc.gdb')
Block_VA = gpd.read_file('../../Data/GDB states/tlgdb_2021_a_51_va.gdb')

In [3]:
## block group geodataframe MD, DC, VA

gdb_MD_BG = gpd.read_file('../../Data/GDB states/tlgdb_2021_a_24_md.gdb',driver='FileGDB', layer='Block_Group')

gdb_DC_BG = gpd.read_file ('../../Data/GDB states/tlgdb_2021_a_11_dc.gdb', driver='FileGDB', layer='Block_Group')

gdb_VA_BG = gpd.read_file ('../../Data/GDB states/tlgdb_2021_a_51_va.gdb', driver='FileGDB', layer='Block_Group')


In [4]:
SLD_data.rename(columns = {'GEOID10':"GEOID"}, inplace = True)

In [5]:
SLD_data = SLD_data.to_crs('EPSG:4326')
gdb_MD_BG = gdb_MD_BG.to_crs('EPSG:4326')
gdb_DC_BG = gdb_DC_BG.to_crs('EPSG:4326')
gdb_VA_BG = gdb_VA_BG.to_crs('EPSG:4326')

In [6]:
SLD_MD = gpd.clip(SLD_data, gdb_MD_BG)
SLD_DC = gpd.clip(SLD_data, gdb_DC_BG)
SLD_VA = gpd.clip(SLD_data, gdb_VA_BG)

In [7]:
## merge three SLD for MD, DC, VA ###
import pandas as pd
SLD_1 = pd.merge(SLD_MD, SLD_DC, how='outer')
SLD = pd.merge(SLD_1, SLD_VA, how='outer')

In [8]:
SLD['fullarea'] = SLD.area

In [9]:
SLD = SLD.to_crs('EPSG:4326')

In [10]:
from geopandas.tools import overlay

intersect_SLD_Boundaries_p5, intersect_SLD_Boundaries_p75 = overlay_wks(SLD)

# intersect_SLD_Boundaries = walkshedbuffer.overlay(SLD, how="intersection")


In [11]:
#find the partial area of the intersected boundaries to get the proportional area

intersect_SLD_Boundaries_p5['partialarea'] = intersect_SLD_Boundaries_p5.area
intersect_SLD_Boundaries_p75['partialarea'] = intersect_SLD_Boundaries_p75.area

# intersect_SLD_Boundaries['Name_1']=intersect_SLD_Boundaries['Name_1'].str.replace(' : 0 - 2640','')
# intersect_SLD_Boundaries['Name_1']=intersect_SLD_Boundaries['Name_1'].str.replace(' : 0 - 22.4525758392805','')

In [13]:
## 97 unique metro stations ####

# intersect_SLD_Boundaries['Name_1'].unique()

array(['ADDISON ROAD-SEAT PLEASANT', 'CAPITOL HEIGHTS', 'ANACOSTIA',
       'CONGRESS HEIGHTS', 'ARCHIVES-NAVY MEMORIAL-PENN QUARTER',
       'ARLINGTON CEMETERY', 'CAPITOL SOUTH', 'EASTERN MARKET',
       'FARRAGUT NORTH', 'FARRAGUT WEST', 'FEDERAL CENTER SW',
       'FEDERAL TRIANGLE', 'FOGGY BOTTOM-GWU', 'GALLERY PL-CHINATOWN',
       'JUDICIARY SQUARE', "L'ENFANT PLAZA", 'MCPHERSON SQUARE',
       'METRO CENTER', 'PENTAGON', 'SMITHSONIAN', 'UNION STATION',
       'NAVY YARD-BALLPARK', 'WATERFRONT',
       'MT VERNON SQ 7TH ST-CONVENTION CENTER', 'ROSSLYN',
       'PENTAGON CITY', 'BALLSTON-MU', 'VIRGINIA SQUARE-GMU', 'CLARENDON',
       'BENNING ROAD', 'MINNESOTA AVE', 'BETHESDA', 'MEDICAL CENTER',
       'BRADDOCK ROAD', 'KING ST-OLD TOWN', 'BRANCH AVE', 'BROOKLAND-CUA',
       'RHODE ISLAND AVE-BRENTWOOD', 'GEORGIA AVE-PETWORTH', 'CHEVERLY',
       'DEANWOOD', 'LANDOVER', 'COURT HOUSE', 'CLEVELAND PARK',
       'WOODLEY PARK-ZOO/ADAMS MORGAN', 'VAN NESS-UDC', 'TENLEYTOWN-AU',
   

In [12]:
## total street intersection density (D3B)
Sum_intersection_dens_bufstation_p5 = intersect_SLD_Boundaries_p5.groupby(['Name_1'])[['D3B']].apply(sum)
Sum_intersection_dens_bufstation_p75 = intersect_SLD_Boundaries_p75.groupby(['Name_1'])[['D3B']].apply(sum)

In [13]:
totalarea_station_p5 = intersect_SLD_Boundaries_p5.groupby(['Name_1'])[['fullarea']].apply(sum)
partialarea_station_p5 = intersect_SLD_Boundaries_p5.groupby(['Name_1'])[['partialarea']].apply(sum)
multiplier_p5 = partialarea_station_p5['partialarea']/totalarea_station_p5['fullarea']

# repeat for 0.75 miles
totalarea_station_p75 = intersect_SLD_Boundaries_p75.groupby(['Name_1'])[['fullarea']].apply(sum)
partialarea_station_p75 = intersect_SLD_Boundaries_p75.groupby(['Name_1'])[['partialarea']].apply(sum)
multiplier_p75 = partialarea_station_p75['partialarea']/totalarea_station_p75['fullarea']

In [14]:
proportion_density_walkstation_p5 = Sum_intersection_dens_bufstation_p5.multiply(multiplier_p5, axis="index")
proportion_density_walkstation_p5.rename(columns={'D3B': 'proportional_street_density_p5'}, inplace=True)

proportion_density_walkstation_p75 = Sum_intersection_dens_bufstation_p75.multiply(multiplier_p75, axis="index")
proportion_density_walkstation_p75.rename(columns={'D3B': 'proportional_street_density_p75'}, inplace=True)

In [15]:
proportion_density_walkstation_p5.reset_index().to_csv("output/proportion_density_walkstation_p5.csv",index=False)
proportion_density_walkstation_p75.reset_index().to_csv("output/proportion_density_walkstation_p75.csv",index=False)